* TF (Term Frequency) :
    * => 특정 문서에서 단어들의 빈도
    * => 특정 문서에서 특정 단어의 등장횟수 / 특정 문서의 총 단어수 

<img class="progressiveMedia-image js-progressiveMedia-image" data-src="https://cdn-images-1.medium.com/max/1600/1*HM0Vcdrx2RApOyjp_ZeW_Q.png" src="https://cdn-images-1.medium.com/max/1600/1*HM0Vcdrx2RApOyjp_ZeW_Q.png">

* IDF (Inverse Data Frequency) : 
    * => 모든 문서들을 통틀어 희귀한 단어.. 
    * log ( 총 문서 개수 / 이 단어를 포함한 문서 개수 )
    * 희귀한 단어일수록 idf 증가.
* __stopwords__ : 자주 등장하지만 중요하지 않은 단어 (다른 문서에서도 자주 등장함..) ex) the, will, you.. 

<img class="progressiveMedia-image js-progressiveMedia-image" data-src="https://cdn-images-1.medium.com/max/1600/1*A5YGwFpcTd0YTCdgoiHFUw.png" src="https://cdn-images-1.medium.com/max/1600/1*A5YGwFpcTd0YTCdgoiHFUw.png">

* TF - IDF : 
    * tf * idf 둘을 곱한 것 
    * 이 문서에서 자주 등장한 단어이면서, 다른 문서에서는 자주 등장하지 않은 단어일경우 값이 커짐.
    
<img class="progressiveMedia-image js-progressiveMedia-image" data-src="https://cdn-images-1.medium.com/max/1600/1*nSqHXwOIJ2fa_EFLTh5KYw.png" src="https://cdn-images-1.medium.com/max/1600/1*nSqHXwOIJ2fa_EFLTh5KYw.png">

<img class="graf-image" data-image-id="1*q2tRgjV_J-MLvnhwNAl0KQ.png" data-width="337" data-height="86" src="https://cdn-images-1.medium.com/max/1600/1*q2tRgjV_J-MLvnhwNAl0KQ.png">


In [2]:
import json
from pathlib import Path  # Path클래스 임포트
import re  # 특수문자 제거
from functools import cmp_to_key


#내림차순 정렬을 위한 함수객체들
def cmp(l,r):
    if l>r:
        return 1
    elif l==r :
        return 0
    else:
        return -1
def key_func(l,r):
    if l[1]==r[1]:
        return cmp(l[0],r[0])
    return -cmp(l[1],r[1])

#dictionary => 내림차순 정렬된 list of tuple으로 만들어서 리턴
def sortDict(bow):
    word_list=[] #리스트에 (단어, 횟수) 꼴의 튜플을 넣을 것

    for i,k in bow.items() :
        word_list.append((i,k))
    word_list.sort(key=lambda x:x[1], reverse=True) #횟수를 기준으로 내림차순 정렬
    word_list.sort(key=cmp_to_key(key_func))# 횟수는같을때, 단어를 기준으로 정렬
    return word_list
#list of tuple을 출력
def displayTupList(tupList, num, withVal=False):  # 디폴트는 내림차순, 값 없이..
    counter = 0
    for tup in tupList:
        if tup[1] == 0 or counter == num:
            break
        else:
            if withVal:
                print(tup[0], " : ", tup[1], "  ", end="")
            else:
                print(tup[0], "   ", end="")
            counter += 1
    print()
#어떤 특수기호를 제거할지?
#일단은 알파벳,숫자,--, % 빼고 대체
def delSymbol(feature):
    str = re.sub('[^A-Za-z0-9-]+', ' ', feature)  # 알파벳,숫자,-,% 빼고 다 지움. # str = re.sub('[^A-Za-z0-9-%]+', ' ', feature)
    str = re.sub('[0-9]+', 'ㅠ', str)  # 숫자를 ㅠ 로 바꿈 (나중에 덩어리째로 지우기)
    while (str.find("--") != -1):  # - 가 두개이상 있는경우엔 지움
        str = str.replace("--", " ")
    return str.strip('-')  # -가 사이에 낀게 아니라 겉에남아있는경우엔 제거

#버릴단어인지?
# 일단은 -- , % 만 남거나, 숫자만 남으면 버림
def isSemantic(word):
    if word == '' or word == '-' or word == '%' or word.isdigit() or word.find("ㅠ") != -1:
        return False
    else:
        return True

#경로에 있는 모든 id.json 파일의 features로 bag of word(딕셔너리)를 만들어 리턴
def makeBow(path) :
    for item in path.iterdir():
        if(item.is_dir()): #안보이는 디렉토리가 있다.. 주의 ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        features = itemJson["features"]

        for feature in features:
            feature = delSymbol(feature)  # 1. 특수기호, 이모티콘을 띄어쓰기로 대체
            for word in feature.split():  # 2. 띄어쓰기 단위로 분리
                word = word.lower().strip('-')  # 3. 겉에 남은 -  제거!
                if not isSemantic(word):  # 4. 무의미한 word는 버림
                    continue
                if word in bow:
                    bow[word] += 1
                else:
                    bow[word] = 1
    return bow


if __name__ == "__main__" :
    #bagOfWord 만들고, 저장--------------------------------------------
    path = Path('/home/jovyan/work/크롤링/featureContainer')
    bow = {}
    bow = makeBow(path)
    # 횟수를 0으로 초기화 한, dictionary 틀
    json.dump(bow, open(f'/home/jovyan/work/분석/bagOfWord' + '.json', 'w+')) 

In [1]:
#미쳤다 ;;;;; 
dicA = {1:0, 2:0, 3:0}
dicB = dicA 

dicB[1] = 3
print(dicA)

{1: 3, 2: 0, 3: 0}


In [18]:
#특정 문서(features)하나 만의 bag of word를 만들고, 단어개수 리턴.
def makeSmallBow(features) :
    smallBow = {}
    for word, count in bow.items():
            smallBow[word] = 0
    counter = 0 #단어개수
    for feature in features:
        feature = delSymbol(feature) # 1. 특수기호, 이모티콘을 띄어쓰기로 대체
        for word in feature.split(): #2. 띄어쓰기 단위로 분리
            word = word.lower().strip('-')  #3. 겉에 남은 -  제거!
            if not isSemantic(word): # 4. 무의미한 word는 버림
                continue
            smallBow[word] += 1
            #print(word, "카운트 1 증가")
            counter += 1
    return smallBow, counter

#문서 하나의 tf를 계산 (dictionary)                
def computeTF(bow, features):  
    smallBow, numOfWord = makeSmallBow(features)  # eachBow : 특정 상품의 word count dict
    # numOfWord : 특정 상품의 word 개수
    tf = {}  # 단어별 tf를 계산해서 넣음
    for word, count in smallBow.items():
        if (numOfWord == 0):  # feature 없는 상품..
            break
        tf[word] = count / float(numOfWord)
    return tf

#모든 tf  가지고, idf 를 계산 (dictionary)
def computeIDF(tfs) :
    import math
    idf = {}
    for word, count in bow.items():
            idf[word] = 0
    N = len(tfs)  # 모든 상품 개수
    print("N -> ", N)
    for ID, tf in tfs.items(): 
        for word, val in tf.items():
            if val > 0:
                idf[word] += 1  # 일단은 "word : word를 가진 상품개수" 꼴의 사전 완성..
    for word, val in idf.items():
        idf[word] = math.log10(N / float(val))
    return idf

#문서 하나의 tf-idf 계산 (dictionary)
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val * idfs[word]
    return tfidf

#path 에 있는 모든 id.json 파일을 가지고, tf를 구하여 "ID" : tf   꼴의 딕셔너리 리턴
def computeAllTF(path):
    count = 0
    for item in path.iterdir():
        if(item.is_dir()): #안보이는 디렉토리가 있다.. 주의 ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        features = itemJson["features"]
        ID = itemJson["id"]
        
        tf = computeTF(bow, features)
        tfDict[ID] = tf
    return tfDict

if __name__ == "__main__":
    # ---------------메인1(tf 전부 계산)---------------
    #tfDict = {} #tf 들을 저장
    #tfDict = computeAllTF(path)
    # ______메인2(idf 계산)___________
    #idf = computeIDF(tfDict)
    ##displayTupList(sortDict(idf), -1, withVal=True)
    #json.dump(idf, open(f'/home/jovyan/work/분석/idf' + '.json', 'w+')) 

    #----------------메인2(tf - idf 전부 계산)-------------
    index = 0
    tfidfDict = {} #  {ID : tf-idf}
    for item in path.iterdir():
        if(item.is_dir()): #안보이는 디렉토리가 있다.. 주의 ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        features = itemJson["features"]
        ID = itemJson["id"]
        tfidf = computeTFIDF(tfDict[ID], idf)
        tfidfDict[ID] = tfidf
        if index == 10:
            break
        print("<features>")
        print(features)
        print("< TF >")
        displayTupList(sortDict(tfDict[ID]), -1)
        print(" <TF - IDF > ")
        displayTupList(sortDict(tfidf), -1)
        #print(tfidf)
        index += 1
        if index == 5:
            break
    #json.dump(tfidf, open(f'/home/jovyan/work/분석/TF-IDFs' + '.json', 'w+')) 

<features>
['❤ COZY MATERIAL - 70% Cotton and 30% Acrylic . Super Soft , Stretchy and Light Weight Material for Comfy and Warm.', '❤ FEATURES - Knitted Heart , Color Block , Cute Sweaters , Ribbed Edges , Crew Neck , Leisure Style , Loose Fitting, Love Heart Pattern.', '❤ OCCASIONS - Suitable for Valentine Day , Holiday , Casual Wear , Work , Party , Home ; Perfect for Fall and Winter .', "❤ SIZE REFERENCE - Model Wears a Size MEDIUM and She Is 5'5'', 130Lbs . If You Prefer A Looser Fit We Suggest Ordering A Size Up !", '❤ 100% MONEY BACK GUARANTEE:Within 30 Days, If Your New Sweater Does Not Make You Perfectly Happy , We Will Be Glad To Process Full Refund for You. To Us, Your Satisfaction Is Always Our Top Priority.']
< TF >
and    for    a    size    you    heart    if    is    material    to    we    your    acrylic    always    back    be    block    casual    color    comfy    cotton    cozy    crew    cute    day    days    does    edges    fall    features    fit    fitting    